In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

In [13]:
import tensorflow as tf
import numpy as np
import pandas as pd
import altair as alt
from path_explain.path_explainer_tf import PathExplainerTF

In [14]:
baseline = np.random.randn(1000, 10)
inputs = np.random.randn(50, 10)

In [15]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(10, dtype=tf.float32))
model.add(tf.keras.layers.Dense(5, activation=tf.keras.activations.relu, use_bias=True))
model.add(tf.keras.layers.Dense(1, activation=None, use_bias=False))

In [40]:
explainer = PathExplainerTF(model)

In [47]:
current_alphas = explainer._sample_alphas(num_samples=100, use_expectation=True)
interactions = explainer._single_interaction(inputs[0], baseline,
                              current_alphas, num_samples=100, batch_size=50,
                              use_expectation=True, output_index=None,
                              interaction_index=None)

In [48]:
interactions = explainer.interactions(inputs, baseline,
                     batch_size=50, num_samples=100,
                     use_expectation=True, output_indices=[0] * len(inputs),
                     verbose=False, interaction_index=0)

In [46]:
interactions.shape

(50, 10)